In [ ]:
import pandas as pd
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import time
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

import csv

In [ ]:
navigator = webdriver.Chrome(ChromeDriverManager().install())
navigator.get(os.getenv('URL'))

In [ ]:
def click(path, type='xpath'):
    if type == 'xpath':
        navigator.find_element(By.XPATH, path).click()
    if type == 'name':
        navigator.find_element(By.NAME, path).click()

In [ ]:
def translate_title(title):
    if title == 'Ano':
        return 'YEAR'
    if title == 'Km':
        return 'KM'
    if title == 'Combustível':
        return 'FUEL'  
    if title == 'Cor':
        return 'COLOR'
    if title == 'Estimativa de mercado':
        return 'FIPE'
    if title == 'Pátio':
        return 'YARD'
    if title == 'Retomada':
        return 'RESUME-TYPE'
    if title == 'Tipo':
        return 'TYPE'  
    if title == 'Possui Chave':
        return 'HAS-KEY'

In [ ]:
def get_element_text(navigator, path):
    value = navigator.find_element(By.XPATH, os.getenv(path)).text
    return value

In [ ]:
def get_vehicle_data():
    title = []
    value = []
    for i in range(1,10):
        try:     
            result = navigator.find_element(By.XPATH,f"/html/body/div[2]/div/div[1]/main/div[2]/div[2]/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[{i}]/div/div").text.split('\n')
            title.append(result[0])                
            value.append(result[1])
        except:
            pass

    data = {translate_title(title[i]): value[i] for i in range(len(title))}
    return data

##### Get Title

In [ ]:
auction_title = get_element_text(navigator, 'AUCTION-TITLE').replace('/','_')
auction_title

In [ ]:
total_results = int(get_element_text(navigator, 'TOTAL-RESULTS').split(' ')[0])
total_results

#### Click on car

In [ ]:
time.sleep(5)
click(os.getenv('FIRST-CAR'))
time.sleep(5)

In [ ]:
def write_csv(name, cars):
    with open(f'{name}.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = cars[0].keys())
        writer.writeheader()
        writer.writerows(cars)

In [ ]:
# def get_information(navigator):
#     data = {'BATCH': find_element(navigator, 'BATCH'),
#         'VEHICLE': find_element(navigator, 'VEHICLE'),
#         'KM':find_element(navigator, 'KM'),
#         'YEAR': find_element(navigator, 'YEAR'),
#         'FUEL': find_element(navigator, 'FUEL'),
#         'COLOR': find_element(navigator, 'COLOR'),
#         'FIPE': find_element(navigator, 'FIPE'),
#         'YARD': find_element(navigator, 'YARD'),
#         'RESUME-TYPE': find_element(navigator, 'RESUME-TYPE'),
#         'TYPE':find_element(navigator, 'TYPE'),
#         'HAS-KEY': find_element(navigator, 'HAS-KEY'),
#         'BANK':find_element(navigator, 'BANK'),
#         }
#     return data

In [ ]:
def get_information(navigator):
    car_info = get_vehicle_data()
    full_car_info = {'BATCH': get_element_text(navigator, 'BATCH'),
        'VEHICLE': get_element_text(navigator, 'VEHICLE'),
        'BANK':get_element_text(navigator, 'BANK'),
        'KM':'',
        'YEAR': '',
        'FUEL': '',
        'COLOR': '',
        'FIPE': '',
        'YARD': '',
        'RESUME-TYPE': '',
        'TYPE':'',
        'HAS-KEY': ''
        }
    full_car_info.update(car_info)
    return full_car_info

In [ ]:
cars = []

for _ in range(1, total_results+1):
    try:
        data = get_information(navigator)
        cars.append(data)
        time.sleep(7)
        click(os.getenv('NEXT_VEHICLE')) 
    except:
        print('erro')
        click(os.getenv('NEXT_VEHICLE')) 

In [ ]:
write_csv(name = auction_title, cars = cars)